In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies_df= movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [5]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [6]:
type(movies_df['genres'][0])
# str 타입인 것을 확인할 수 있다.

str

### genres, keywords 칼럼들의 str형태를 list형태로 바꿔주기

In [7]:
from ast import literal_eval  # 문자열 파싱 라이브러리

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [8]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [9]:
type(movies_df['genres'][0])
# str타입에서 list타입으로 바뀐 것을 확인할 수 있다.

list

### list 내 여러개 딕셔너리의 name키에 해당하는 값들을 리스트로 변환

In [10]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

In [11]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


## -> 데이터 전처리 끝

## 장르 컨텐츠 유사도 측정

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
#Vectorizer

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [22]:
movies_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [23]:
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

In [24]:
# 장르 매트릭스 생성 1
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2)) #min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [25]:
print(genre_mat)

  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
  (1, 0)	1
  (1, 16)	1
  (1, 124)	1
  (1, 24)	1
  (1, 125)	1
  (2, 0)	1
  (2, 16)	1
  (2, 1)	1
  (2, 64)	1
  (2, 20)	1
  (3, 0)	1
  (3, 64)	1
  (3, 90)	1
  (3, 234)	1
  (3, 4)	1
  (3, 68)	1
  :	:
  (4796, 90)	1
  (4796, 234)	1
  (4796, 106)	1
  (4796, 144)	1
  (4797, 234)	1
  (4797, 153)	1
  (4797, 157)	1
  (4798, 0)	1
  (4798, 64)	1
  (4798, 234)	1
  (4798, 4)	1
  (4798, 78)	1
  (4799, 44)	1
  (4799, 214)	1
  (4799, 58)	1
  (4800, 90)	1
  (4800, 44)	1
  (4800, 214)	1
  (4800, 104)	1
  (4800, 50)	1
  (4800, 250)	1
  (4800, 182)	1
  (4800, 251)	1
  (4800, 229)	1
  (4802, 80)	1


In [26]:
# 장르 매트릭스 생성 2
count_vect2 = CountVectorizer(min_df=1, ngram_range=(1,1)) #min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat2 = count_vect2.fit_transform(movies_df['genres_literal'])
print(genre_mat2.shape)

(4803, 22)


In [27]:
print(genre_mat2)

  (0, 0)	1
  (0, 1)	1
  (0, 8)	1
  (0, 17)	1
  (0, 9)	1
  (1, 0)	1
  (1, 1)	1
  (1, 8)	1
  (2, 0)	1
  (2, 1)	1
  (2, 4)	1
  (3, 0)	1
  (3, 4)	1
  (3, 6)	1
  (3, 18)	1
  (4, 0)	1
  (4, 1)	1
  (4, 17)	1
  (4, 9)	1
  (5, 0)	1
  (5, 1)	1
  (5, 8)	1
  (6, 2)	1
  (6, 7)	1
  (7, 0)	1
  :	:
  (4792, 18)	1
  (4792, 12)	1
  (4792, 15)	1
  (4793, 6)	1
  (4794, 18)	1
  (4794, 3)	1
  (4794, 12)	1
  (4795, 6)	1
  (4796, 17)	1
  (4796, 9)	1
  (4796, 6)	1
  (4796, 18)	1
  (4797, 18)	1
  (4797, 10)	1
  (4798, 0)	1
  (4798, 4)	1
  (4798, 18)	1
  (4799, 3)	1
  (4799, 16)	1
  (4800, 6)	1
  (4800, 3)	1
  (4800, 16)	1
  (4800, 19)	1
  (4800, 13)	1
  (4802, 5)	1


In [30]:
# CountVectorizer로 학습
count_vect2 = CountVectorizer(min_df=0, ngram_range=(1,1)) #min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat2 = count_vect2.fit_transform(movies_df['genres_literal'])
print(genre_mat2.shape)

(4803, 22)


### 코사인 유사도(cosine_similarity)이용해서 영화별 유사도 계산

In [31]:
# 코사인 유사도에 의해 4803개 영화 각각 유사한 영화들이 계산됨
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [32]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 영화를 앞에서부터 순서대로 보여줌
# 0번째 영화의 경우 유사도 순서 : 0번, 3494번, 813번, ..., 2401 순서
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 전체를 -1칸 간격으로
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [33]:
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

# 장르 코사인 유사도에 의해 영화를 추천하는 함수

### 함수 분석

In [34]:
# 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
title_movie = movies_df[movies_df['title'] == 'The Godfather']
title_movie

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
3337,238,The Godfather,"[Drama, Crime]",8.4,5893,143.659698,"[italy, love at first sight, loss of father, p...","Spanning the years 1945 to 1955, a chronicle o...",Drama Crime


In [35]:
# title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
# sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
title_index = title_movie.index.values
title_index

array([3337], dtype=int64)

In [36]:
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes

array([[2731, 1243, 3636, 1946, 2640, 4065, 1847, 4217,  883, 3866]],
      dtype=int64)

In [37]:
# 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
# dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
similar_indexes = similar_indexes.reshape(-1)
print(similar_indexes)

[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]


In [38]:
movies_df.iloc[similar_indexes]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
2731,240,The Godfather: Part II,"[Drama, Crime]",8.3,3338,105.792936,"[italo-american, cuba, vororte, melancholy, pr...",In the continuing saga of the Corleone crime f...,Drama Crime
1243,203,Mean Streets,"[Drama, Crime]",7.2,345,17.002096,"[epilepsy, protection money, secret love, mone...","A small-time hood must choose from among love,...",Drama Crime
3636,36351,Light Sleeper,"[Drama, Crime]",5.7,15,6.063868,"[suicide, drug dealer, redemption, addict, exi...",A drug dealer with upscale clientele is having...,Drama Crime
1946,11699,The Bad Lieutenant: Port of Call - New Orleans,"[Drama, Crime]",6.0,326,17.339852,"[police brutality, organized crime, policeman,...","Terrence McDonagh, a New Orleans Police sergea...",Drama Crime
2640,400,Things to Do in Denver When You're Dead,"[Drama, Crime]",6.7,85,6.932221,"[father son relationship, bounty hunter, boat,...",A mafia film in Tarantino style with a star-st...,Drama Crime
4065,364083,Mi America,"[Drama, Crime]",0.0,0,0.039007,"[new york state, hate crime]",A hate-crime has been committed in a the small...,Drama Crime
1847,769,GoodFellas,"[Drama, Crime]",8.2,3128,63.654244,"[prison, based on novel, florida, 1970s, mass ...","The true story of Henry Hill, a half-Irish, ha...",Drama Crime
4217,9344,Kids,"[Drama, Crime]",6.8,279,13.291991,"[puberty, first time]",A controversial portrayal of teens in New York...,Drama Crime
883,640,Catch Me If You Can,"[Drama, Crime]",7.7,3795,73.944049,"[con man, biography, fbi agent, overhead camer...","A true story about Frank Abagnale Jr. who, bef...",Drama Crime
3866,598,City of God,"[Drama, Crime]",8.1,1814,44.356711,"[male nudity, street gang, brazilian, photogra...",Cidade de Deus is a shantytown that started du...,Drama Crime


### 실제 함수

In [39]:
def find_sim_movie_ver1(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)    
    # 2차원 데이터를 1차원으로 변환
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

### 영화 Gotfather와 장르가 유사한 영화 10개 추천

In [40]:
similar_movies = find_sim_movie_ver1(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'genres', 'vote_count']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average,genres,vote_count
2731,The Godfather: Part II,8.3,"[Drama, Crime]",3338
1243,Mean Streets,7.2,"[Drama, Crime]",345
3636,Light Sleeper,5.7,"[Drama, Crime]",15
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,"[Drama, Crime]",326
2640,Things to Do in Denver When You're Dead,6.7,"[Drama, Crime]",85
4065,Mi America,0.0,"[Drama, Crime]",0
1847,GoodFellas,8.2,"[Drama, Crime]",3128
4217,Kids,6.8,"[Drama, Crime]",279
883,Catch Me If You Can,7.7,"[Drama, Crime]",3795
3866,City of God,8.1,"[Drama, Crime]",1814


In [41]:
# 평점 기준으로 정렬(sort_values)해보니 평가횟수(vote_count)가 낮은 것들도 포함됨. 평가횟수 반영한 평점(가중평점)이 필요함
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893
